In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# import torch
# from transformers import BertTokenizer, BertForSequenceClassification
# import numpy as np

# # Load pre-trained GloVe embeddings
# glove_path = '/kaggle/input/glove-text-file/glove.6B.100d.txt'  # Path to your GloVe embeddings file
# embedding_matrix[index] = np.reshape(glove_embeddings[word], (1, -1))

# glove_embeddings = {}

# with open(glove_path, 'r', encoding='utf-8') as f:
#     for line in f:
#         values = line.strip().split(' ')
#         word = values[0]
#         embeddings = np.asarray(values[1:], dtype=np.float32)
#         glove_embeddings[word] = embeddings

# # Load pre-trained BERT model and tokenizer
# model_name = 'bert-base-uncased'
# tokenizer = BertTokenizer.from_pretrained(model_name)
# model = BertForSequenceClassification.from_pretrained(model_name, num_labels=2)

# # Initialize BERT model's embedding layer with GloVe embeddings
# vocab_size = tokenizer.vocab_size
# embedding_dim = model.config.hidden_size

# embedding_matrix = np.zeros((vocab_size, embedding_dim))

# for word, index in tokenizer.get_vocab().items():
#     if word in glove_embeddings:
#         embedding_matrix[index] = glove_embeddings[word]

# model.bert.embeddings.word_embeddings.weight.data = torch.tensor(embedding_matrix)

# # Example user query
# user_query = "machine learning"

# # Tokenize user query
# query_tokens = tokenizer.tokenize(user_query)
# query_ids = tokenizer.convert_tokens_to_ids(query_tokens)
# query_ids = tokenizer.build_inputs_with_special_tokens(query_ids)

# # Convert query to tensor
# query_tensor = torch.tensor([query_ids])

# # Forward pass through the BERT model
# outputs = model(query_tensor)

# # Get the predicted probabilities and predicted label
# probs = torch.softmax(outputs[0], dim=1)
# predicted_label = torch.argmax(probs, dim=1).item()

# # Convert label index to label text
# label_map = {0: "Not Relevant", 1: "Relevant"}  # Modify as per your task
# predicted_label_text = label_map[predicted_label]

# # Print results
# print("User Query:", user_query)
# print("Predicted Label:", predicted_label_text)
# print("Predicted Probabilities:", probs.tolist())


In [ ]:
!pip install --upgrade keras 


In [ ]:
!pip install keras

In [ ]:
import keras
print(keras.__version__) 


In [ ]:
import re
import string
import numpy as np 
import random
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from plotly import graph_objs as go
import plotly.express as px
import plotly.figure_factory as ff
from collections import Counter

from PIL import Image
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

from tqdm import tqdm
import os
import spacy
from spacy.util import compounding
from spacy.util import minibatch

from collections import defaultdict

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.initializers import Constant
from tensorflow.keras.layers import (LSTM, 
                                      Embedding, 
                                      BatchNormalization,
                                      Dense, 
                                      TimeDistributed, 
                                      Dropout, 
                                      Bidirectional,
                                      Flatten, 
                                      GlobalMaxPool1D)
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam

from sklearn.metrics import (precision_score, 
                             recall_score, 
                             f1_score, 
                             classification_report,
                             accuracy_score)


In [ ]:
test_df = pd.read_csv("/kaggle/input/querycodeexp/test_data.csv",nrows=500 )
test_df.head()

In [ ]:
train_df = pd.read_csv("/kaggle/input/querycodeexp/train_data.csv", nrows=1000)
train_df.head()

In [ ]:
balance_counts = train_df.groupby('docstring')['docstring'].agg('count').values
balance_counts

In [ ]:
balance_counts = test_df.groupby('url')['url'].agg('count').values
balance_counts

In [ ]:
import plotly.graph_objects as go
fig = go.Figure()
fig.add_trace(go.Bar(
    x=['docstring'],
    y=[balance_counts[0]],
    name='docstring',
    text=[balance_counts[0]],
    textposition='auto',
))
fig.add_trace(go.Bar(
    x=['url'],
    y=[balance_counts[1]],
    name='url',
    text=[balance_counts[1]],
    textposition='auto',
))
fig.update_layout(
    title='<span style="font-size:32px; font-family:Times New Roman">Dataset distribution by target</span>'
)
fig.show()

In [ ]:
# Special thanks to https://www.kaggle.com/tanulsingh077 for this function
def clean_docstring(docstring):
    '''Make docstring lowercase, remove docstring in square brackets,remove links,remove punctuation
    and remove words containing numbers.'''
    docstring = str(docstring).lower()
    docstring = re.sub('\[.*?\]', '', docstring)
    docstring = re.sub('https?://\S+|www\.\S+', '', docstring)
    docstring = re.sub('<.*?>+', '', docstring)
    docstring = re.sub('[%s]' % re.escape(string.punctuation), '', docstring)
    docstring = re.sub('\n', '', docstring)
    docstring = re.sub('\w*\d\w*', '', docstring)
    return docstring  

In [ ]:
train_df['docstring_clean'] = train_df['docstring'].apply(clean_docstring)
train_df.head()

In [ ]:
test_df['docstring_clean'] = test_df['docstring'].apply(clean_docstring)
test_df.head()

In [ ]:
stop_words = stopwords.words('english')
more_stopwords = ['u', 'im', 'c']
stop_words = stop_words + more_stopwords

def remove_stopwords(docstring):
    docstring = ' '.join(word for word in docstring.split(' ') if word not in stop_words)
    return docstring
    
test_df['docstring_clean'] = test_df['docstring_clean'].apply(remove_stopwords)
train_df['docstring_clean'] = train_df['docstring_clean'].apply(remove_stopwords)

In [ ]:
stemmer = nltk.SnowballStemmer("english")

def stemm_docstring(docstring):
    docstring = ' '.join(stemmer.stem(word) for word in docstring.split(' '))
    return docstring

In [ ]:
test_df['docstring_clean'] = test_df['docstring_clean'].apply(stemm_docstring)


In [ ]:
train_df['docstring_clean'] = train_df['docstring_clean'].apply(stemm_docstring)

In [ ]:
def preprocess_data(docstring):
    # Clean puntuation, urls, and so on
    docstring = clean_docstring(docstring)
    # Remove stopwords
    docstring = ' '.join(word for word in docstring.split(' ') if word not in stop_words)
    # Stemm all the words in the sentence
    docstring = ' '.join(stemmer.stem(word) for word in docstring.split(' '))
    
    return docstring

In [ ]:
test_df['docstring_clean'] = test_df['docstring_clean'].apply(preprocess_data)


In [ ]:
train_df['docstring_clean'] = train_df['docstring_clean'].apply(preprocess_data)

In [ ]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
le.fit(train_df['url'])


train_df['url_encoded'] = le.transform(train_df['url'])
train_df.head()

In [ ]:
le = LabelEncoder()
le.fit(test_df['url'])


test_df['url_encoded'] = le.transform(test_df['url'])
test_df.head()

In [ ]:
test_x = test_df['docstring_clean']
test_y = test_df['url_encoded']

print(len(test_x), len(test_y))

In [ ]:
train_x = train_df['docstring_clean']
train_y = train_df['url_encoded']

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

# instantiate the vectorizer
vect = CountVectorizer()
vect.fit(train_x)


In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

# instantiate the vectorizer
vect = CountVectorizer()
vect.fit(test_x)

In [ ]:
x_test_dtm = vect.transform(test_x)
x_train_dtm = vect.transform(train_x)

In [ ]:
vect_tunned = CountVectorizer(stop_words='english', ngram_range=(1,2), min_df=0.1, max_df=0.7, max_features=100)

In [ ]:
from sklearn.feature_extraction.text import TfidfTransformer

tfidf_transformer = TfidfTransformer()

tfidf_transformer.fit(x_train_dtm)
x_train_tfidf = tfidf_transformer.transform(x_train_dtm)

x_train_tfidf

In [ ]:
texts = train_df['docstring_clean']
target = train_df['url_encoded']

In [ ]:
# Calculate the length of our vocabulary
word_tokenizer = Tokenizer()
word_tokenizer.fit_on_texts(texts)

vocab_length = len(word_tokenizer.word_index) + 1
vocab_length

In [ ]:
def embed(corpus): 
    return word_tokenizer.texts_to_sequences(corpus)

longest_train = max(texts, key=lambda sentence: len(word_tokenize(sentence)))
length_long_sentence = len(word_tokenize(longest_train))

train_padded_sentences = pad_sequences(
    embed(texts), 
    length_long_sentence, 
    padding='post'
)

train_padded_sentences

In [ ]:
embeddings_dictionary = dict()
embedding_dim = 100

# Load GloVe 100D embeddings
with open('/kaggle/input/codetextexp/glove.6B.100d.txt') as fp:
    for line in fp.readlines():
        records = line.split()
        word = records[0]
        vector_dimensions = np.asarray(records[1:], dtype='float32')
        embeddings_dictionary [word] = vector_dimensions

# embeddings_dictionary

In [ ]:
 #Now we will load embedding vectors of those words that appear in the
# Glove dictionary. Others will be initialized to 0.

embedding_matrix = np.zeros((vocab_length, embedding_dim))

for word, index in word_tokenizer.word_index.items():
    embedding_vector = embeddings_dictionary.get(word)
    if embedding_vector is not None:
        embedding_matrix[index] = embedding_vector
        
embedding_matrix

In [ ]:
# Create a Multinomial Naive Bayes model
from sklearn.naive_bayes import MultinomialNB
nb = MultinomialNB()

# Train the model
nb.fit(x_train_dtm, train_y)

In [ ]:
# Make class anf probability predictions
y_pred_class = nb.predict(x_test_dtm)
y_pred_prob = nb.predict_proba(x_test_dtm)[:, 1]

In [ ]:
# # calculate accuracy of class predictions
# from sklearn import metrics
# import seaborn as sns
# import matplotlib.pyplot as plt
# from sklearn.metrics import confusion_matrix
# import numpy as np

# confusion = confusion_matrix(test_y, y_pred_class)
# print(confusion)

# print(metrics.accuracy_score(test_y, y_pred_class))

# conf_matrix(metrics.confusion_matrix(test_y, y_pred_class))

In [ ]:
# # Calculate AUC
# metrics.roc_auc_score(test_y, y_pred_prob, multi_class='ovr')

In [ ]:
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.pipeline import Pipeline

pipe = Pipeline([('bow', CountVectorizer()), 
                 ('tfid', TfidfTransformer()),  
                 ('model', MultinomialNB())])

In [ ]:
# Fit the pipeline with the data
pipe.fit(train_x, train_y)

y_pred_class = pipe.predict(test_x)

# print(metrics.accuracy_score(test_y, y_pred_class))

# conf_matrix(metrics.confusion_matrix(test_y, y_pred_class))

In [ ]:
import xgboost as xgb

pipe = Pipeline([
    ('bow', CountVectorizer()), 
    ('tfid', TfidfTransformer()),  
    ('model', xgb.XGBClassifier(
        learning_rate=0.1,
        max_depth=7,
        n_estimators=80,
        use_label_encoder=False,
        eval_metric='auc',
        # colsample_bytree=0.8,
        # subsample=0.7,
        # min_child_weight=5,
    ))
])

In [ ]:
from sklearn import metrics


In [ ]:
# Fit the pipeline with the data
pipe.fit(train_x, train_y)

y_pred_class = pipe.predict(test_x)
y_pred_train = pipe.predict(train_x)

print('Train: {}'.format(metrics.accuracy_score(train_y, y_pred_train)))
print('Test: {}'.format(metrics.accuracy_score(test_y, y_pred_class)))

# conf_matrix(metrics.confusion_matrix(test_y, y_pred_class))

**BERT**

In [ ]:
# import torch
# from transformers import BertTokenizer, BertModel
# import numpy as np

# # Load pre-trained GloVe embeddings
# glove_path = '/kaggle/input/glove-text-file/glove.6B.100d.txt'  # Path to your GloVe embeddings file
# glove_embeddings = {}

# with open(glove_path, 'r', encoding='utf-8') as f:
#     for line in f:
#         values = line.strip().split(' ')
#         word = values[0]
# #         embeddings = np.asarray(values[1:], dtype=np.float32)
#         glove_embeddings[word] = embeddings

# # Load pre-trained BERT model and tokenizer
# model_name = 'bert-base-uncased'
# tokenizer = BertTokenizer.from_pretrained(model_name)
# model = BertModel.from_pretrained(model_name)

# # Initialize projection layer
# projection = torch.nn.Linear(100, 768)

# # Example input
# input_text = "This is an example sentence."

# # Tokenize input text
# input_ids = tokenizer.encode(input_text, add_special_tokens=True)
# input_ids = torch.tensor(input_ids).unsqueeze(0)  # Add batch dimension

# # Convert GloVe embeddings to tensor
# embeddings_tensor = torch.tensor([glove_embeddings[word] for word in tokenizer.get_vocab() if word in glove_embeddings])

# # Project GloVe embeddings to match BERT hidden size
# projected_embeddings = projection(embeddings_tensor)

# # Initialize BERT model's embedding layer with projected GloVe embeddings
# model.embeddings.word_embeddings.weight.data = projected_embeddings

# # Forward pass through the model
# outputs = model(input_ids)

# # Get the contextualized word embeddings for each token
# contextual_embeddings = outputs.last_hidden_state

# # Print the contextualized word embeddings
# print("Input Text:", input_text)
# print("Contextualized Word Embeddings:", contextual_embeddings)


In [ ]:
!pip install transformers

In [ ]:
!pip install --upgrade transformers

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ModelCheckpoint

import transformers
from tqdm.notebook import tqdm
from tokenizers import BertWordPieceTokenizer

In [ ]:
try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
    
except:
    strategy = tf.distribute.get_strategy()
    
print('Number of replicas in sync: ', strategy.num_replicas_in_sync)

In [ ]:
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

def bert_encode(data, maximum_length) :
    input_ids = []
    attention_masks = []

    for text in data:
        encoded = tokenizer.encode_plus(
            text, 
            add_special_tokens=True,
            max_length=maximum_length,
            pad_to_max_length=True,

            return_attention_mask=True,
        )
        input_ids.append(encoded['input_ids'])
        attention_masks.append(encoded['attention_mask'])
        
    return np.array(input_ids),np.array(attention_masks)

In [ ]:
texts = test_df['docstring_clean']
target = test_df['url_encoded']

train_input_ids, train_attention_masks = bert_encode(texts,60)

In [ ]:
import tensorflow as tf
from tensorflow.keras.optimizers import Adam

def create_model(bert_model):
    
    input_ids = tf.keras.Input(shape=(60,),dtype='int32')
    attention_masks = tf.keras.Input(shape=(60,),dtype='int32')

    output = bert_model([input_ids,attention_masks])
    output = output[1]
    output = tf.keras.layers.Dense(32,activation='relu')(output)
    output = tf.keras.layers.Dropout(0.2)(output)
    output = tf.keras.layers.Dense(1,activation='sigmoid')(output)
    
    model = tf.keras.models.Model(inputs = [input_ids,attention_masks],outputs = output)
    model.compile(Adam(lr=1e-5), loss='binary_crossentropy', metrics=['accuracy'])
    return model

In [ ]:
from transformers import TFBertModel
bert_model = TFBertModel.from_pretrained('bert-large-uncased')

In [ ]:
model = create_model(bert_model)
model.summary()

In [ ]:
model

In [ ]:
# import matplotlib.pyplot as plt

# # Extract loss and accuracy values from the training history
# loss = history['loss']
# val_loss = history['val_loss']
# accuracy = history['accuracy']
# val_accuracy = history['val_accuracy']

# # Plot the learning curves
# plt.plot(loss, label='Training Loss')
# plt.plot(val_loss, label='Validation Loss')
# plt.plot(accuracy, label='Training Accuracy')
# plt.plot(val_accuracy, label='Validation Accuracy')
# plt.xlabel('Epochs')
# plt.ylabel('Metrics')
# plt.title('Learning Curves')
# plt.legend()
# plt.show()


In [ ]:
# plot_learning_curves(history, [['loss', 'val_loss'],['accuracy', 'val_accuracy']])

In [ ]:
from keras.models import load_model

# Save the model
model.save('model.h5')


